# Spark SQL

```
v1.0.0 
Benjamin Harder
1/5/2016
Python 2.7.11
Learning Spark
```
Spark SQL is a extension of Spark which allows for SQL-like queries to be performed against RDDs within memory.  This can be done against native files, or directly to HDFS or HBase - in this case they would be 'HQL-like' queries.  
In this example, I will demonstrate interactive data analysis using text files, Spark SQL, dataframes, and functional programming.  Spark SQL jobs will typically involved a few key steps, these include:


**Intro Examples** 
1. Connect to the environment
2. Create a Spark Context
3. Create a Spark SQL context
4. Read in the file to be analyzed
5. Transform it into a data frame
6. Begin to query that dataframe using Spark SQL.

Detailed documentation can be found on the Apache website [here:](http://spark.apache.org/docs/latest/sql-programming-guide.html) - note: this is for Spark 1.5.2

Spark process and actions can be viewed at http://localhost:4040/jobs/

Lastly - when using Hive the best practice would be to avoid the 4th and 5th steps by connecting to Hive tables directly.  

**Advanced Example**

In [ ]:
import getspark #custom script that activates the environment
from IPython.display import Image
from pyspark import SparkContext 
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.sql import Row

In [ ]:
sc = SparkContext() #Initialize a Spark Context
sqlContext = SQLContext(sc)

As with previous examples, we will begin by reading in the text file for the click data.  This version is in CSV format.  After it is read in as an rdd, the next step is to split it by its delimiter, then remove the header and check out the finalized and headerless rdd.  This is a typical data munging step when dealing with RDDs. 

In [ ]:
rdd = sc.textFile(r"C:\Spark\clickinfo.csv")

In [ ]:
#Split it by its delimiter
rdd = rdd.map(lambda line: line.split(",")) #split it up by comma -transformation

In [ ]:
#Strip out the header
header = rdd.first() #extract header
data = rdd.filter(lambda x:x !=header) #review the headerless rdd

In [ ]:
#Check out your fancy new rdd
data.take(5)

### Why go through the trouble?

* Schema = table + columns + types
* Column names to index
* Leverage SQL and relational theory

### Why not schemas and SQL?

* They make your data structure
* Fragility

When reviewing our schema, one of the columns has a boolean for male == 1 and female == 0.  To ease interpretability for reporting, we can find a replace the values as male and female respectively.  Note: currently only strings exist in the rdd, in the next step they will be redefined as specific ints, floats, strings etc.  The row function with Spark dataframes is helpful because it allows you to programmatically define the schema up front.  

One of the great advantages of Spark is that it is a computing environment and not just a query language.  This means that you can write functions and pass them directly into Spark.  In this case, all of the values with the dataframe are integers which represent factors.  Using Python it is possible to write a collection of functions which loop over the RDD and replace ints with strings.

Using the 'map' function loops the code over all of the rows in the RDD.  'Row' tells Spark to assign a header placeholder to the dataframe, along with defining the schema for that particular column.  Finally, 'toDF()' passes the RDD to a dataframe object.  I recommend that RDD is passed to a dataframe at the earliest opportunity, as they are more performant (especially with Python) than operating on standard RDDS.  

In [ ]:
#Instead of reading in the sql context directly, we can define the rows uniquely in the dataframe and define the schema
def genmap(gender):
    if gender == '0':
        return "Female"
    else:
        return "Male"
    
def clickmap(clicks):
    if clicks == '0':
        return "No_Click"
    else: 
        return "Click"
    
def signedin(clicks):
    if clicks == '0':
        return "Not_Signed_In"
    else: 
        return "Signed_In"

In [ ]:
clicksmappd = data.map(lambda line: Row(user_id = str(line[0]), 
                              clicks = clickmap(line[1]), 
                              impression=int(line[2]), 
                              signedin=signedin(line[3]))).toDF()

Now with our dataframe created, we can review the schema and take a topline sample of the dataframe.  Once we have confirmed that the schema matches our desired format the next step is to register a temporary table to be able to we can then execute SQL queries against the data as you would with any normal SQL query. 

In [ ]:
clicksmappd.printSchema() # This maps the Schema

In [ ]:
clicksmappd.show(5) # Shows a snippet of the data frame

In [ ]:
clicksmappd.registerTempTable("clickinfo")

As a quick review, we have performed all of the necessary operations to get a base csv file into a tabular format to perform Spark SQL queries.  This involved:

1. Reading in the base csv file
2. Stripping out the header
3. Creating functions which looped over the RDD to replace numeric values with strings to improve interpretability.  
4. Mapping the headerless RDD to a dataframe
5. Registering a temporary table against the dataframe for Spark SQL
6. Running and Spark SQL query

In [ ]:
sqlContext.sql("""SELECT clicks, signedin, SUM(impression) as impressions 
                  FROM clickinfo 
                  GROUP BY clicks, signedin 
                  ORDER BY SUM(impression) DESC""").show()

Next you will read in the second csv file that contains gender information about the users, and join that to the clicks dataframe using spark sql.  The point of this is to show how joins are performed within Spark SQL. 

In [ ]:
rdd = sc.textFile(r"C:\Spark\userinfo.csv")
rdd = rdd.map(lambda line: line.split(",")) #split it up by comma -transformation
header = rdd.first() #extract header
data = rdd.filter(lambda x:x !=header) #review the headeress rdd
usersmappd = data.map(lambda line: Row(user_id = str(line[0]), 
                                     gender = genmap(line[1]))).toDF()

In [ ]:
usersmappd.registerTempTable("userinfo")

In [ ]:
joindf = sqlContext.sql("""SELECT clickinfo.user_id, 
                    clickinfo.impression, 
                    clickinfo.clicks, 
                    clickinfo.signedin, 
                    userinfo. gender
                FROM clickinfo, userinfo
                WHERE clickinfo.user_id = userinfo.user_id
                """)

In [ ]:
joindf.show(10)
joindf.registerTempTable("joinedtable")

In [ ]:
report = sqlContext.sql("""SELECT gender, SUM(impression) as impressions 
                  FROM joinedtable 
                  GROUP BY gender 
                  ORDER BY SUM(impression) DESC""")

In [ ]:
report.show(5)

In order to conduct interactive visualizations, it is neccessary to move data from a Spark dataframe to the local machine.  No ways currently exist which allow for visualizations to be performed on large datasets within Spark.  The best way in python to do this is via Pandas and Matplotlib or Seaborne.  the toPandas() function takes the Spark dataframe and moves it into a Pandas dataframe.  It can also be saved as a csv from pandas as well.

In [ ]:
pdrept = report.toPandas()

In [ ]:
pdrept.to_csv(r"...Desktop\FancyReport.csv", index=False)

In [ ]:
import matplotlib.pyplot as plot #Import Matplotlib
%matplotlib inline 

In [ ]:
pdrept.set_index(['gender']).plot(kind='bar',title="Total Impressions By Gender")